In [1]:
import os
import pandas as pd
import requests
import json
from pathlib import Path
from datetime import datetime
import time
import chardet

In [2]:
current_path = Path(os.getenv('WORKING_DIR')) if os.getenv('WORKING_DIR') else Path().absolute()
current_path

PosixPath('/Users/geoffreyaldebert/etalab/projects/analysis/notebooks/irve-v2')

In [3]:
own_datasets = ['54231d4a88ee38334b5b9e1d', '5448d3e0c751df01f85d0572']
def parse_api(url):
    r = requests.get(url)
    data = r.json()
    nb_pages = int(data['total']/data['page_size'])+1
    arr = []
    for i in range(1,nb_pages+1):
        r = requests.get(url+"&page="+str(i))
        data = r.json()
        for dataset in data['data']:
            for res in dataset['resources']:
                if dataset['id'] not in own_datasets: #dataset Etalab
                    if 'format=csv' in res['url']:
                        filename = res['url'].split('/')[-3] + '.csv'
                    else:
                        filename = res['url'].split('/')[-1]
                    ext = filename.split('.')[-1]
                    obj = {}
                    obj['dataset_id'] = dataset['id']
                    obj['dataset_slug'] = dataset['slug']
                    obj['dataset_page'] = dataset['page']
                    obj['resource_id'] = res['id']
                    obj['resource_title'] = res['title']
                    obj['resource_last_modified'] = res['last_modified']
                    if ext != 'csv':
                        obj['error_type'] = "wrong-file-format"
                    else:
                        if not dataset['organization'] and not dataset['owner']:
                            obj['error_type'] = "orphan-dataset"
                        else:
                            obj['organization_or_owner'] = dataset['organization']['slug'] if dataset['organization'] else dataset['owner']['slug']
                            obj['error_type'] = None
                    arr.append(obj)
    df = pd.DataFrame(arr)
    return df

## Récupération via tags

In [4]:
url = "https://www.data.gouv.fr/api/1/datasets/?tag=irve"

In [5]:
df = parse_api(url)

## Récupération via metadata schema

In [6]:
url = "https://www.data.gouv.fr/api/1/datasets/?schema=etalab/schema-irve"

In [7]:
df2 = parse_api(url)

## Récupération via search

In [8]:
url = "https://www.data.gouv.fr/api/1/datasets/?q=recharge+v%C3%A9hicules+%C3%A9lectriques"
df3 = parse_api(url)
url = "https://www.data.gouv.fr/api/1/datasets/?q=irve"
df4 = parse_api(url)

## Concaténation et dédoublonnage

In [9]:
df = pd.concat([df, df2, df3, df4], ignore_index=True)

In [10]:
df = df.drop_duplicates(subset=['resource_id'], keep='first')

In [11]:
df.shape

(539, 8)

In [12]:
df[df['error_type'].isna()].shape

(337, 8)

# Téléchargement resources

In [13]:
data_path = current_path / 'data' / datetime.now().strftime('%Y%m%d')
data_path.mkdir(parents=True, exist_ok=True)


In [14]:
total_resources = 0
dl_resources = 0

downloaded = []

for index,row in df[df['error_type'].isna()].iterrows():
    rurl = "https://www.data.gouv.fr/fr/datasets/r/"+row['resource_id']
    r = requests.get(rurl, allow_redirects=True) 
    
    p = Path(data_path) / row['dataset_slug']
    p.mkdir(exist_ok=True)
    written_filename = '%s.csv' % (row['resource_id'])
                                       
    with open('%s/%s' % (p, written_filename), 'wb') as f:
        dl_resources += 1
        f.write(r.content)
        downloaded.append(row)
        print('✅ downloaded file [%s] %s' % (row['resource_title'], rurl))

✅ downloaded file [20210201-declaration-bornes-exadys.csv] https://www.data.gouv.fr/fr/datasets/r/52f99086-21f3-460d-bd76-8e0c1218bff1
✅ downloaded file [donnees-territoriales-pour-le-diagnostic-des-schemas-directeurs-irve-v21-05-31.csv] https://www.data.gouv.fr/fr/datasets/r/bd23b47a-2bba-4ad2-8cbe-0a52b021194e
✅ downloaded file [data.gouv-gite-lassalle-le-refrus.csv] https://www.data.gouv.fr/fr/datasets/r/f6ce7b48-8c18-4075-b87f-ac07c62b1106
✅ downloaded file [data.gouv-sarl-opti-chez-soi.csv] https://www.data.gouv.fr/fr/datasets/r/cdb4d934-9088-42e6-8d6b-56768357df18
✅ downloaded file [Monfichier] https://www.data.gouv.fr/fr/datasets/r/6eccc279-7f0a-4658-a61f-1be9e442cdfd
✅ downloaded file [data.gouv-gite-lassalle-le-rouget.csv] https://www.data.gouv.fr/fr/datasets/r/19f357e2-98f9-47ef-9118-2772223f6eec
✅ downloaded file [Monfichier] https://www.data.gouv.fr/fr/datasets/r/988f4026-c149-49ab-ac77-244097f674ee
✅ downloaded file [Monfichier] https://www.data.gouv.fr/fr/datasets/r/a1fa6

✅ downloaded file [irve-ionity-20210709-datagouv.csv] https://www.data.gouv.fr/fr/datasets/r/696b4a10-6181-4dcc-b096-c0c824362091
✅ downloaded file [data-vague-2.csv] https://www.data.gouv.fr/fr/datasets/r/fc85a64c-8097-41c8-ab82-e91df3b3a3c9
✅ downloaded file [irve-reveo-20210409.csv] https://www.data.gouv.fr/fr/datasets/r/8d5e8e1a-e46a-4118-b7a2-da99593f8edf
✅ downloaded file [irve-te90-20200302.csv] https://www.data.gouv.fr/fr/datasets/r/5585604e-0ee9-406b-9998-2b7e7a57f291
✅ downloaded file [mobive-20210315.csv] https://www.data.gouv.fr/fr/datasets/r/7216f0e3-c362-4c7b-9619-517e8797b1bf
✅ downloaded file [mobive-20200707.csv] https://www.data.gouv.fr/fr/datasets/r/784df802-601b-491e-a7c5-cf68d7e61b5d
✅ downloaded file [Fichier CSV] https://www.data.gouv.fr/fr/datasets/r/fadb4829-859a-4c92-a66b-125e6567bcee
✅ downloaded file [Fichier CSV] https://www.data.gouv.fr/fr/datasets/r/bf7824b4-a1c3-4c14-8341-9e946ccfd5a8
✅ downloaded file [irve-wiiizcasa-20210217.csv] https://www.data.gouv.

✅ downloaded file [IRVE_SDE29_20180404.csv] https://www.data.gouv.fr/fr/datasets/r/ef3b67f9-915c-4252-b67e-b2a1f70e7831
✅ downloaded file [IRVE_SDE29_20170822.csv] https://www.data.gouv.fr/fr/datasets/r/04d40b0d-dc78-4a60-a12e-53db2d7d3389
✅ downloaded file [IRVE_SDE29_20170515.csv] https://www.data.gouv.fr/fr/datasets/r/7cddde99-9481-4bb8-a659-ccaad5243e11
✅ downloaded file [IRVE_SDE29_20170126.csv] https://www.data.gouv.fr/fr/datasets/r/c72e84d4-b640-4d68-a0b7-997ecdb0ffa7
✅ downloaded file [SYANE_26/03/20_IRVE_Bornes de recharge] https://www.data.gouv.fr/fr/datasets/r/51f7595a-4ceb-4e46-8490-bbf14478d66c
✅ downloaded file [irve-wiizcacpl-20200518.csv] https://www.data.gouv.fr/fr/datasets/r/4477b75b-f5b6-495b-8588-221cad8798bc
✅ downloaded file [irve-wiizcapl-20190129.csv] https://www.data.gouv.fr/fr/datasets/r/08d2bbe1-6194-4b2a-bd0c-715e9621802d
✅ downloaded file [irve-sdehg-20190528.csv] https://www.data.gouv.fr/fr/datasets/r/bad9974a-56c9-45fd-bb7b-528fed9d007c
✅ downloaded file 

✅ downloaded file [Infrastructures de Recharge pour Véhicules Électriques - Paris Aéroports] https://www.data.gouv.fr/fr/datasets/r/f63cf765-8aa2-47d1-ba7e-5e3d2828daf7
✅ downloaded file [IRVE_SDEY_YONNE_20170519.csv] https://www.data.gouv.fr/fr/datasets/r/e8b3dd76-d39e-4016-80b6-92384f056390
✅ downloaded file [etalab-schema-irve-bluewash1.csv] https://www.data.gouv.fr/fr/datasets/r/9d704fdb-cb98-4361-8d5f-d539f8636c09
✅ downloaded file [etalab-schema-irve-tambour.csv] https://www.data.gouv.fr/fr/datasets/r/5f52c41c-d7d9-4e67-b67d-eff4eee70899
✅ downloaded file [irve-ccphva-20200429.csv] https://www.data.gouv.fr/fr/datasets/r/ee6c78d6-76fc-4308-b8cb-b307712bf832
✅ downloaded file [etalab-schema-irve-blueselfstockage92-copie.csv] https://www.data.gouv.fr/fr/datasets/r/6f82dcf8-e093-451a-ad4a-5a8399263bff
✅ downloaded file [etalab-schema-irve-spi-fermetures.csv] https://www.data.gouv.fr/fr/datasets/r/08e36510-bc95-49d9-a92c-cf36a9ffc871
✅ downloaded file [etalab-schema-irve-bluewash1.csv

✅ downloaded file [etalab-schema-irve-aclea.csv] https://www.data.gouv.fr/fr/datasets/r/e3004f2e-599a-4844-bf0c-ec1f0d7f5c7f
✅ downloaded file [etalab-schema-irve-emendo-2128.csv] https://www.data.gouv.fr/fr/datasets/r/2420e873-88ee-45d0-8aa1-5e4927ae2991
✅ downloaded file [antibes-irve.csv] https://www.data.gouv.fr/fr/datasets/r/810f064b-3bd1-48db-b435-bb3f337f1c2a
✅ downloaded file [IRVE_sdesm_20210609.csv] https://www.data.gouv.fr/fr/datasets/r/6d189577-af67-4d72-ba60-73fa62fd35fc
✅ downloaded file [Export au format CSV] https://www.data.gouv.fr/fr/datasets/r/e0bd6928-78b9-420c-a877-e44ec2e2ded5
✅ downloaded file [Export au format CSV] https://www.data.gouv.fr/fr/datasets/r/1fc79025-df85-402b-82e4-561ce95f3fb3
✅ downloaded file [Export au format CSV] https://www.data.gouv.fr/fr/datasets/r/2346c6fb-1feb-4544-907c-644edb83f177
✅ downloaded file [IRVE_SIDELC.csv] https://www.data.gouv.fr/fr/datasets/r/85a9385f-c8bd-4d02-aafa-2bea3ec16ccc
✅ downloaded file [IRVE_SIEIL.csv] https://www.d

✅ downloaded file [BORNE IRVE AC2R] https://www.data.gouv.fr/fr/datasets/r/1dd6d0ca-2c2c-43c0-832c-11b5d4e32868
✅ downloaded file [dat-gouv3.csv] https://www.data.gouv.fr/fr/datasets/r/f5b1a158-97cc-44a7-975f-b59e17fa8eba
✅ downloaded file [dat-gouv3.csv] https://www.data.gouv.fr/fr/datasets/r/b801cbfe-bc67-42d0-a5e8-b6b96d12d494
✅ downloaded file [dat-gouv3.csv] https://www.data.gouv.fr/fr/datasets/r/0d971ee6-7dff-497e-a950-0d2b0662b96c
✅ downloaded file [irve-provost-halluin-20210462.csv] https://www.data.gouv.fr/fr/datasets/r/2a6d728e-4f45-46f4-b92b-79a44c487c08
✅ downloaded file [irve-overchem-20210628.csv] https://www.data.gouv.fr/fr/datasets/r/88254472-85e8-423f-b7e7-ca432d4ec214
✅ downloaded file [irve-provost-neuville-20210402.csv] https://www.data.gouv.fr/fr/datasets/r/c743618f-39a2-4515-8854-8f75529fdcc0
✅ downloaded file [irve-sgar-20190304.csv] https://www.data.gouv.fr/fr/datasets/r/eadaeee8-c76b-4f26-99b0-3ba75e684558
✅ downloaded file [irve-irve-54-20181019-cd-54.csv] htt

## Analyse via Validata

In [15]:
listOfFiles = list()
for (dirpath, dirnames, filenames) in os.walk(data_path):
    listOfFiles += [os.path.join(dirpath, file) for file in filenames]

In [16]:
len(listOfFiles)

337

In [17]:
def recalibrating_v1(path):
    modified = 0
    df = pd.read_csv(path,sep=None, engine='python',dtype=str)
    #Traitements v1
    columns_mapping = {
        'n_amenageur': ['nom_amenageur', 'n_amenageu'],
        'n_operateur': ['n_operateu','nom_operateur'],
        'n_enseigne': ['nom_enseigne'],
        'id_station': ['id'],
        'n_station': ['nom_station','station'],
        'ad_station': ['adresse_station'],
        'code_insee': ['insee'],
        'Xlongitude': ['longitude_wsg84'],
        'Ylatitude': ['latitude_wsg84'],
        'nbre_pdc': ['nbre_borne'],
        'id_pdc': ['n° borne'],
        'puiss_max': ['puissance_max','puissance_maximale'],
        'type_prise': ['type_connecteur','typ_charge'],
        'acces_recharge': ['modalité d\'accès à la borne','acces_rech'],
        'accessibilité': ['accessibilitã©','accessibilite','accessibilit�','accessibilit','accessibilit'],
        'observations': ['obs'],
        'date_maj': ['date_mise_a_jour']
    }
    for col in df.columns:
        for key in columns_mapping:
            if col in columns_mapping[key]:
                df = df.rename(columns={col:key})
                modified = 1
    df.to_csv(path,index=False)

In [18]:
schema_v2_url = 'https://schema.data.gouv.fr/schemas/etalab/schema-irve/2.0.0/schema.json'
schema_v1_url = 'https://schema.data.gouv.fr/schemas/etalab/schema-irve/1.0.3/schema.json'
validata_api = 'https://api.validata.etalab.studio/validate'

In [19]:
arr = []
cpt = 0
for f in listOfFiles:
    try:
        cpt +=1
        print("Processing "+str(cpt)+" : "+f.replace('.csv','').replace('csv/','').split("/")[-1])
        obj = {}
        obj['resource_id'] = f.replace('.csv','').replace('csv/','').split("/")[-1]
        obj['local_file'] = f
        file = {'file': open(f,'rb')}
        data = {'schema': schema_v2_url}
        time.sleep(1)
        r = requests.post(validata_api, files=file, data=data)
        if(r.json()['report']['valid']):
            obj['version'] = '2.0.0'
        else:
            recalibrating_v1(f)
            time.sleep(1)
            file = {'file': open(f,'rb')}
            data = {'schema': schema_v1_url}
            r = requests.post(validata_api, files=file, data=data)
            if(r.json()['report']['valid']):
                obj['version'] = '1.0.3'
            else:
                obj['error_type'] = 'validation'
        arr.append(obj)
    except:
        obj = {}
        obj['resource_id'] = f.replace('.csv','').replace('csv/','').split("/")[-1]
        obj['local_file'] = f
        obj['error_type'] = 'read'
        arr.append(obj)
        pass


Processing 1 : 0d971ee6-7dff-497e-a950-0d2b0662b96c
Processing 2 : 198eaabe-5f9c-4f91-998a-b3c3cb615e52
Processing 3 : c8d32a3e-d3ec-490e-904f-e48c9aa3eaa1
Processing 4 : 76fdb0c1-1d8c-4a43-a709-eb7eeb1d674e
Processing 5 : c9460bd6-e50b-4dd6-9851-75fccb228c7b
Processing 6 : 88e0d165-57a9-4762-8b89-86321757c943
Processing 7 : 4b07173d-9e6c-4895-ad62-c28fc1276778
Processing 8 : f267e0f2-cd70-41c3-80cd-8f3bf1d8c881
Processing 9 : 5d3ddd2e-16f6-40ba-8b90-648e828a26a6
Processing 10 : 85d4e170-dd22-4592-bc0e-870e9ec9835b
Processing 11 : 005c0524-4403-4d30-bdc4-c7a13e81c9a7
Processing 12 : a7171c9f-eb3b-4b96-9709-993522810144
Processing 13 : 9e75a85e-255e-4f4d-8bf5-713587c1ba5e
Processing 14 : ca662511-1ea0-4358-822d-b982fe7b2a4f
Processing 15 : a214acff-191d-4ddb-8ff5-0c9bfa5a68d0
Processing 16 : 30c1dae9-1976-4061-93e5-5059cacea5bd
Processing 17 : 988f4026-c149-49ab-ac77-244097f674ee
Processing 18 : abef512f-da52-4178-bc10-4cd06ef07c35
Processing 19 : 96874829-54c3-4d16-b2f3-5302e4eed962
Pr

Processing 155 : 7fcf7267-d050-4006-b8fc-897445c8bfef
Processing 156 : c6bf56ac-2a26-4384-bef9-2f7846dfe7cc
Processing 157 : e86ae03b-069c-4961-a65b-52b8a6788d1c
Processing 158 : add8fd4c-9444-4b5e-b9cb-81c7c9dfa57e
Processing 159 : 7b33cfe5-534e-478c-b2a1-bd3dd0e38132
Processing 160 : 446e191b-a51e-4678-9788-d28faa8a3600
Processing 161 : d612ca97-ab92-4916-85f2-d298294c61b1
Processing 162 : 8fd4119e-52dd-489d-9316-343a4f817591
Processing 163 : 42abe450-6454-4f16-ae92-9fb7735d77f7
Processing 164 : 94db1f41-9c5a-4de1-85cf-bd037492ccb3
Processing 165 : ab4ef6a8-f9cc-42a6-bb5c-1ed9d5e4e545
Processing 166 : e6107e71-deae-4860-8cff-f9e60b46aa7f
Processing 167 : ad6eea4a-9ceb-4c9d-8b25-e9cea40c49f0
Processing 168 : 98dca738-7d14-452d-b134-a3e42c70edc9
Processing 169 : 59fc69d9-25df-43fc-adb4-8de966640478
Processing 170 : 368764b2-d377-4714-a0f7-79e3e0dfccbc
Processing 171 : bad69a76-507a-4e33-8b8c-3c74e387608f
Processing 172 : 957b6ddc-681e-477d-8e9a-779d731ccd6b
Processing 173 : 1bf801fa-43

Processing 307 : 3dcf1e9f-98cd-4b96-9c96-2123df1dd155
Processing 308 : 7bd44410-7412-4c56-850f-ccafcf405ecc
Processing 309 : 391d0d67-8bba-4d7b-865d-a26f847d6fc2
Processing 310 : 4cbb9f6c-11f6-4243-a40b-c773fb605e5d
Processing 311 : c8de760a-b234-404b-8163-23f085751b48
Processing 312 : 08dc2820-d795-4bfc-8b7b-caa6e86b78ab
Processing 313 : 0f82a01c-b44f-4d51-bdfb-9be5e6d42627
Processing 314 : d72232ff-dd3b-4b3a-8881-873450e3ddcd
Processing 315 : ac843faf-3be1-4d43-b69f-4e9f2caa8a66
Processing 316 : 05ab8a52-2976-489c-a6db-01bcbefa4e92
Processing 317 : 6ee579c7-f9d2-4395-8dc1-b0420ad50479
Processing 318 : 92731582-cb1b-4035-8509-0ad41d872e9e
Processing 319 : c791cedd-e30f-4f6b-b11f-803faff21420
Processing 320 : 52f99086-21f3-460d-bd76-8e0c1218bff1
Processing 321 : 8b32bef4-cab9-4b91-a737-0dea26a5238d
Processing 322 : a22f7243-998a-4e8d-8322-b428040a4223
Processing 323 : 2e99ea46-eee8-4dfb-8f2b-98f2032704da
Processing 324 : e5942621-061d-4764-a5ac-332695feda84
Processing 325 : 87c60c59-07

In [20]:
proc = pd.DataFrame(arr)

## Dédoublonnage

In [21]:
df = pd.merge(df,proc,on='resource_id',how='left')

In [22]:
df.error_type_x.update(df.error_type_y)

In [23]:
df = df.rename(columns={'error_type_x':'error_type'})
df = df.drop(columns=['error_type_y'])

In [68]:
def dedoublonnage(df, col):
    arr = []
    df = df.sort_values('last_modified', ascending=False)
    valcounts = pd.DataFrame(df[col].value_counts())
    doublons = valcounts[valcounts[col] > 1].index.to_list()
    if "Non concerné" in doublons: doublons.remove("Non concerné")
    for item in doublons:
        lendf = len(df[df[col] == item].index.to_list())
        for i in range(0,lendf-1):
            obj = {}
            obj['resource_id_keep'] = df[df.index == df[df[col] == item].index.to_list()[0]]['resource_id'].iloc[0]
            obj['last_modified_keep'] = df[df.index == df[df[col] == item].index.to_list()[0]]['last_modified'].iloc[0]
            obj['resource_id_delete'] = df[df.index == df[df[col] == item].index.to_list()[i+1]]['resource_id'].iloc[0]
            obj['last_modified_delete'] = df[df.index == df[df[col] == item].index.to_list()[i+1]]['last_modified'].iloc[0]
            obj[col] = item
            arr.append(obj)
        df = df.drop(df[df[col] == item].index.to_list()[1:])
    return df, arr

In [69]:
versions = df['version'].dropna().unique().tolist()
for v in ["2.0.0"]:
    print(v)
    cpt = 0
    arr = []
    for index,row in df[df['version'] == v].iterrows():
        with open(row['local_file'],'rb') as f:
            encoding = chardet.detect(f.read()).get('encoding')  
            if(encoding == 'Windows-1254'):
                encoding = 'iso-8859-1'

        if cpt == 0:
            dfConso = pd.read_csv(row['local_file'], sep=None, engine="python",dtype=str,encoding=encoding)
            dfConso['last_modified'] = row['resource_last_modified']
            dfConso['resource_id'] = row['resource_id']
            cpt += 1
        else:
            dfinter = pd.read_csv(row['local_file'], sep=None, engine="python",dtype=str,encoding=encoding)
            dfinter['last_modified'] = row['resource_last_modified']
            dfinter['resource_id'] = row['resource_id']
            dfConso = pd.concat([dfConso, dfinter], ignore_index=True)
            # Dédoublonnage here
        dfConso, arrinter = dedoublonnage(dfConso, 'id_pdc_itinerance')
        arr = arr + arrinter

    dfConso = dfConso.drop_duplicates()
    dfConso.to_csv('irve-'+v+'.csv',index=False, encoding="utf-8",na_rep='null')

2.0.0


In [70]:
log2 = pd.DataFrame(arr)

## Publication

In [71]:
log_path = current_path / 'logs'
log_path.mkdir(parents=True, exist_ok=True)
out_path = current_path / 'output'
out_path.mkdir(parents=True, exist_ok=True)

file_output = 'irve-consolidation-v2.0.0-'+datetime.now().strftime('%Y%m%d')+'.csv'
file_log_general = "log-general-"+datetime.now().strftime('%Y%m%d')+".csv"
file_log_dedoublonnage = "log-doublons-"+datetime.now().strftime('%Y%m%d')+".csv"


csv_output_path = out_path / file_output
log_general_path = log_path / file_log_general
log_dedoublonnage_path = log_path / file_log_dedoublonnage

In [72]:
log2.to_csv(log_dedoublonnage_path,index=False)

In [73]:
df['error_type'] = df['error_type'].apply(lambda x: 'no-error' if x == None else x)

In [74]:
df.drop(columns=['local_file']).to_csv(log_general_path,index=False)

In [75]:
dfConso.to_csv(csv_output_path,index=False)

In [76]:
domain = os.getenv('NB_IRVE_DOMAIN', 'www.data.gouv.fr')
domain

'www.data.gouv.fr'

In [ ]:
apikey = os.environ['NB_IRVE_APIKEY']
dataset_id = os.environ['NB_IRVE_DATASET_ID']
latest_resource_id = os.environ['NB_IRVE_LATEST_RESOURCE_ID_V2']
latest_resource_id_log_general = os.environ['NB_IRVE_RESOURCE_ID_LOG_GENERAL']
latest_resource_id_log_dedoublonnage = os.environ['NB_IRVE_RESOURCE_ID_LOG_DEDOUBLONNAGE']

In [78]:

url = 'https://%s/api/1/datasets/%s' % (domain, dataset_id)

obj = {}
obj['title'] = "logs général - consolidation v2.0.0 - "+datetime.now().strftime('%Y%m%d')

with open(log_general_path, 'rb') as csvfile:
    files = {'file': (file_log_general, csvfile.read())}

r = requests.post('%s/resources/%s/upload' % (url, latest_resource_id_log_general), files=files, headers={'x-api-key': apikey})

r = requests.put('%s/resources/%s/' % (url, latest_resource_id_log_general), json=obj, headers={'x-api-key': apikey})
assert r.status_code == 200


obj = {}
obj['title'] = "logs dédoublonnage - consolidation v2.0.0 - "+datetime.now().strftime('%Y%m%d')

with open(log_dedoublonnage_path, 'rb') as csvfile:
    files = {'file': (file_log_dedoublonnage, csvfile.read())}

r = requests.post('%s/resources/%s/upload' % (url, latest_resource_id_log_dedoublonnage), files=files, headers={'x-api-key': apikey})

r = requests.put('%s/resources/%s/' % (url, latest_resource_id_log_dedoublonnage), json=obj, headers={'x-api-key': apikey})
assert r.status_code == 200

In [104]:
url = 'https://%s/api/1/datasets/%s' % (domain, dataset_id)

schema = {
    "name": "etalab/schema-irve",
    "version": "2.0.0"
}
obj = {}
obj['schema'] = schema
obj['title'] = "Dernière version consolidée (v2.0.0 du schéma) - "+datetime.now().strftime('%Y%m%d')
obj['format'] = 'csv'

with open(csv_output_path, 'rb') as csvfile:
    files = {'file': (file_output, csvfile.read())}

r = requests.post('%s/resources/%s/upload/' % (url, latest_resource_id), files=files, headers={'x-api-key': apikey})

r = requests.put('%s/resources/%s/' % (url, latest_resource_id), json=obj, headers={'x-api-key': apikey})
assert r.status_code == 200

